# Pair

Basic examples of pairs, trying to extract fires.
Here the reference product will be the pre-event image, the secondary being the post-event.

## Initialization of the notebook

- Imports
- Loggers
- Paths

In [1]:
from cloudpathlib import AnyPath
from eosets.pair import Pair
from eoreader.bands import NBR
from matplotlib import pyplot as plt
import cartopy.crs as ccrs

In [2]:
# Create logger
import logging
from sertit import logs

logs.init_logger(logging.getLogger("eoreader"), log_lvl=logging.INFO)
logs.init_logger(logging.getLogger("eosets"))

In [3]:
# Get the base paths
data_path = AnyPath(r"/home/ds2_db3/CI/eosets/DATA")

In [4]:
# Get a dictionary of Sentinel-2, Sentinel-3 and Landsat-8 reference and secondary images
s2_paths = {
    "reference_paths": [data_path / "S2A_MSIL1C_20200824T110631_N0209_R137_T29TQE_20200824T150432.zip"],
    "secondary_paths": [data_path / "S2B_MSIL1C_20200908T110619_N0209_R137_T29TQE_20200908T132324.zip"],
}

s3_paths = {
    "reference_paths": [data_path / "S3B_SL_1_RBT____20200824T105515_20200824T105815_20200825T151744_0179_042_322_2340_LN2_O_NT_004.SEN3"],
    "secondary_paths": [data_path / "S3B_SL_1_RBT____20200909T104016_20200909T104316_20200910T161910_0179_043_165_2340_LN2_O_NT_004.SEN3"],
}

l8_paths = {
    "reference_paths": [data_path / "LC08_L1TP_202032_20200828_20200906_02_T1.tar"],
    "secondary_paths": [data_path / "LC08_L1TP_202032_20200929_20201006_02_T1.tar"],
}

all_paths = [s2_paths, s3_paths, l8_paths]

aoi_path = data_path / "Fire_Spain.geojson"

## Creation of the Pairs and loading of dNBR

To create a pair, you just have to give the paths of your products.
The reference and secondary products should overlap.

The goal here will be to extract the `dNBR` band of the pair and display it for every pair.

$$
    dNBR = NBR_{pre} - NBR_{post}

$$
$$
    NBR = \frac{NIR - SWIR}{NIR + SWIR}
$$

In [5]:
for paths in all_paths:
    # Create pair with reference and secondary paths
    pair = Pair(**paths)

    # Load dNBR
    _, _, dnbr_ds = pair.load(
        diff_bands=NBR,
        window=aoi_path,
        pixel_size=60
    )
    dnbr = dnbr_ds[NBR]
    dnbr.attrs["long_name"] = f"dNBR of {pair.condensed_name}"

    # Plot the pairs
    crs = ccrs.UTM(str(dnbr.rio.crs.to_epsg())[-2:])
    dnbr.plot(
        robust=True,
        transform=crs,
        x="x",
        y="y",
        cmap="gist_heat",
        subplot_kws={'projection':crs}
    )
    plt.show()

2024-04-24 15:32:51,732 - [DEBUG] - *** Loading ['NBR'] for 20200824T110631_S2_T29TQE_L1C_150432 ***
2024-04-24 15:32:54,295 - [DEBUG] - Merging bands NBR
2024-04-24 15:32:54,403 - [DEBUG] - Collocating bands
2024-04-24 15:32:54,406 - [DEBUG] - *** Loading ['NBR'] for 20200908T110619_S2_T29TQE_L1C_132324 ***


0...10...20...30...40...50...60...70...80...90...100 - done.


2024-04-24 15:32:56,955 - [DEBUG] - Merging bands NBR
2024-04-24 15:32:57,050 - [DEBUG] - Collocating bands


0...10...20...30...40...50...60...70...80...90...100 - done.


KeyError: "No variable named 'NBR'. Variables on the dataset include ['x', 'y', 'spatial_ref', 'band']"

## Stacking pairs

A pair can also be stacked.

In [ ]:
pair = Pair(**s2_paths)
stack = pair.stack(
    reference_bands=NBR,
    secondary_bands=NBR,
    diff_bands=NBR,
    window=aoi_path,
    pixel_size=60
    )
stack

In [ ]:
stack.plot.imshow(vmin=0, vmax=1, robust=True)